# Using the V3IO Frames Library for High-Performance Data Access 

- [Overview](#frames-overview)
- [Initialization](#frames-init)
- [Working with NoSQL Tables ("kv" Backend)](#frames-kv)
- [Working with Time-Series Databases ("tsdb" Backend)](#frames-tsdb)
- [Working with Streams ("stream" Backend)](#frames-stream)
- [Cleanup](#frames-cleanup)

<a id="frames-overview"></a>
## Overview

[V3IO Frames](https://github.com/v3io/frames) (**"Frames"**) is a multi-model open-source data-access library, developed by Iguazio, which provides a unified high-performance DataFrame API for loading, storing, and accessing data in the data store of the Iguazio Data Science Platform (**"the platform**).
Frames currently supports the NoSQL (key/value), stream, and time-series (TSDB) data models via its "kv", "stream", and "tsdb" backends.

To use Frames, you first need to import the **v3io_frames** library and create and initialize a client object &mdash; an instance of the`Client` class.<br>
The `Client` class features the following object methods for supporting basic data operations:

- `create` &mdash; create a new NoSQL or TSDB table or a stream ("the backend").
- `delete` &mdash; delete the backend.
- `read` &mdash; read data from the backend (as a pandas DataFrame or DataFrame iterator).
- `write` &mdash; write one or more DataFrames to the backend.
- `execute` &mdash; execute a command on the backend. Each backend may support multiple commands.


For a detailed description of the Frames API, see the [Frames documentation](https://github.com/v3io/frames/blob/development/README.md).<br>
For more help and usage details, use the internal API help &mdash; `<client object>.<command>?` in Jupyter Notebook or `print(<client object>.<command>.__doc__)`.<br>
For example, the following command returns information about the read operation for a client object named `client`:
```
client.read?
```

<a id="frames-init"></a>
## Initialization

To use V3IO Frames, first ensure that your platform tenant has a shared tenant-wide instance of the V3IO Frames service.
This can be done by a platform service administrator from the **Services** dashboard page.<br>
Then, import the required libraries and create a Frames client object (an instance of the `Client` class), as demonstrated in the following code, which creates a client object named `client`.

> **Note:**
> - The client constructor's `container` parameter is set to `"users"` for accessing data in the platform's "users" data container.
> - Because no authentication credentials are passed to the constructor, Frames will use the access token that's assigned to the `V3IO_ACCESS_KEY` environment variable.
>   The platform's Jupyter Notebook service defines this variable automatically and initializes it to an access token for the running user of the service.
>   You can pass different credentials by using the constructor's `token` parameter (platform access token) or `user` and `password` parameters (platform username and password).

In [1]:
import pandas as pd
import v3io_frames as v3f
import os

# Create a Frames client
client = v3f.Client("framesd:8081", container="users")

<a id='frames-kv'></a>
## Working with NoSQL Tables ("kv" Backend)

This section demonstrates how to use the `"kv"` Frames backend to write and read NoSQL data in the platform.

- [Initialization](#frames-kv-init)
- [Load Data from Amazon S3](frames-kv-load-data-s3)
- [Write to a NoSQL Table](#frames-kv-write)
- [Read from the Table Using an SQL Query](#frames-kv-read-sql-query)
- [Read from the Table Using the Frames API](#frames-kv-read-frames-api)
  - [Read Using a Single DataFrame](#frames-kv-read-frames-api-single-df)
  - [Read Using a DataFrame Iterator (Streaming)](#frames-kv-read-frames-api-df-iterator)
- [Delete the NoSQL Table](#frames-kv-delete)

<a id="frames-kv-init"></a>
### Initialization

Start out by defining table-path variables that will be used in the tutorial's code examples.<br>
The table path (`table`) is relative to the configured parent data container; see [Write to a NoSQL Table](#frames-kv-write).

In [2]:
# Relative path to the NoSQL table within the parent platform data container
table = os.path.join(os.getenv("V3IO_USERNAME") + "/examples/bank")

# Full path to the NoSQL table for SQL queries (platform Presto data-path syntax);
# use the same data container as used for the Frames client ("users")
sql_table_path = 'v3io.users."' + table + '"'

<a id="frames-kv-load-data-s3"></a>
### Load Data from Amazon S3

Read a file from an Amazon Simple Storage (S3) bucket into a Frames pandas DataFrame.

In [3]:
# Read an AWS S3 file into a DataFrame and show its data and metadata
df = pd.read_csv("https://s3.amazonaws.com/iguazio-sample-data/bank.csv", sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


<a id="frames-kv-write"></a>
### Write to a NoSQL Table

Use the `write` method of the Frames client with the `"kv"` backend to write the data that was read in the previous step to a NoSQL table.<br>
The mandatory `table` parameter specifies the relative table path within the data container that was configured for the Frames client (see the [main initialization](#frames-init) step).
In the following example, the relative table path is set by using the `table` variable that was defined in the ["kv" backend initialization](#frames-kv-init) step.<br>
The `dfs` parameter can be set either to a single DataFrame (as done in the following example) or to multiple DataFrames &mdash; either as a DataFrame iterator or as a list of DataFrames.

In [4]:
out = client.write("kv", table=table, dfs=df)

<a id="frames-kv-read-sql-query"></a>
### Read from the Table Using an SQL Query

You can run SQL queries on your NoSQL table (using Presto) to offload data filtering, grouping, joins, etc. to a scale-out high-speed database engine.

> **Note:** To query a table in a platform data container, the table path in the `from` section of the SQL query should be of the format `v3io.<container name>."/path/to/table"`.
> See [Presto Data Paths](https://www.iguazio.com/docs/tutorials/latest-release/getting-started/fundamentals/#data-paths-presto) in the platform documentation.
> In the following example, the path is set by using the `sql_table_path` variable that was defined in the ["kv" backend initialization](#frames-kv-init) step.
> Unless you changed the code, this variable translates to `v3io.users."<running user>/examples/bank"`; for example, `v3io.users."iguazio/examples/bank"` for user "iguazio".

In [5]:
%sql select * from $sql_table_path where balance > 10000

Done.


loan,education,previous,housing,poutcome,duration,marital,default,balance,month,contact,campaign,y,job,day,age,pdays
no,secondary,0,no,unknown,219,married,no,26452,jul,telephone,2,no,retired,15,75,-1
no,secondary,0,yes,unknown,249,married,no,19317,aug,cellular,1,yes,retired,4,68,-1


<a id="frames-kv-read-frames-api"></a>
### Read from the Table Using the Frames API

Use the `read` method of the Frames client with the `"kv"` backend to read data from your NoSQL table.<br>
The `read` method can return a DataFrame or a DataFrame iterator (a stream), as demonstrated in the following examples.

- [Read Using a Single DataFrame](#frames-kv-read-frames-api-single-df)
- [Read Using a DataFrame Iterator (Streaming)](#frames-kv-read-frames-api-df-iterator)

<a id="frames-kv-read-frames-api-single-df"></a>
#### Read Using a Single DataFrame

The following example uses a single command to read data from the NoSQL table into a DataFrame.

In [6]:
df = client.read(backend="kv", table=table, filter="balance > 20000")
df.head(8)

,age,balance,campaign,contact,day,default,duration,education,housing,job,loan,marital,month,pdays,poutcome,previous,y
index,,,,,,,,,,,,,,,,,
75,75.0,26452.0,2.0,telephone,15.0,no,219.0,secondary,no,retired,no,married,jul,-1.0,unknown,0.0,no


<a id="frames-kv-read-frames-api-df-iterator"></a>
#### Read Using a DataFrame Iterator (Streaming)

The following example uses a DataFrame iterator to stream data from the NoSQL table into multiple DataFrames and allow concurrent data movement and processing.<br>
The example sets the `iterator` parameter to `True` to receive a DataFrame iterator (instead of the default single DataFrame), and then iterates the DataFrames in the returned iterator; you can also use `concat` instead of iterating the DataFrames.

> **Note:** Iterators work with all Frames backends and can be used as input to write functions that support this, such as the `write` method of the Frames client.

In [7]:
dfs = client.read(backend="kv", table=table, filter="balance > 20000", iterator=True)
for df in dfs:
    print(df.head())

        age  balance  campaign    contact   day default  duration  education  \
index                                                                          
75     75.0  26452.0       2.0  telephone  15.0      no     219.0  secondary   

      housing      job loan  marital month  pdays poutcome  previous   y  
index                                                                     
75         no  retired   no  married   jul   -1.0  unknown       0.0  no  


<a id="frames-kv-delete"></a>
### Delete the NoSQL Table

Use the `delete` method of the Frames client with the `"kv"` backend to delete the NoSQL table that was used in the previous steps.

In [8]:
# Delete the `table` NoSQL table
client.delete("kv", table)

<a id='frames-tsdb'></a>
## Working with Time-Series Databases ("tsdb" Backend)

This section demonstrates how to use the `"tsdb"` Frames backend to create a time-series database (TSDB) table in the platform, ingest data into the table, and read from the table (i.e., submit TSDB queries).

- [Initialization](#frames-tsdb-init)
- [Create a TSDB Table](#frames-tsdb-create)
- [Write to the TSDB Table](#frames-tsdb-write)
- [Read from the TSDB Table](#frames-tsdb-read)
- [Delete the TSDB Table](#frames-tsdb-delete)

<a id="frames-tsdb-init"></a>
### Initialization

Start out by defining a TSDB table-path variable that will be used in the tutorial's code examples.<br>
The table path (`tsdb_table`) is relative to the configured parent data container; see [Create a TSDB Table](#frames-tsdb-create).

In [9]:
# Relative path to the TSDB table within the parent platform data container
tsdb_table = os.path.join(os.getenv("V3IO_USERNAME") + "/examples/tsdb_tab")

<a id="frames-tsdb-create"></a>
### Create a TSDB Table

Use the `create` method of the Frames client with the `"tsdb"` backend to create a new TSDB table.<br>
The mandatory `table` parameter specifies the relative table path within the data container that was configured for the Frames client (see the [main initialization](#frames-init) step).
In the following example, the relative table path is set by using the `tsdb_table` variable that was defined in the ["tsdb" backend initialization](#frames-tsdb-init) step.<br>
You can optionally use the `attrs` parameter to provide additional arguments.
For example, you can set the `rate` argument to the TSDB’s metric-samples ingestion rate (`"[0-9]+/[smh]"`; for example, `1/s`); the rate should be calculated according to the slowest expected ingestion rate.

In [10]:
# Create a new TSDB table; ingestion rate = one sample per minute ("1/m")
client.create(backend="tsdb", table=tsdb_table, attrs={"rate": "1/m"})

<a id="frames-tsdb-write"></a>
### Write to the TSDB Table

Use the `write` method of the Frames client with the `"tsdb"` backend to ingest data from a pandas DataFrame into your TSDB table.<br>
The primary-key attribute of platform TSDB tables (i.e., the DataFrame index column) must hold the sample time of the data (displayed as `time` in read outputs).<br>
In addition, TSDB table items (rows) can optionally have sub-index columns (attributes) that are called labels.
You can add labels to TSDB table items in one of two ways; you can also combine these methods:

- Use the `labels` dictionary parameter of the `write` method to add labels to all the written metric-sample table items (DataFrame rows) &mdash; `{"<label>": "<value>"[, "<label>": "<value>", ...]}`.<br>
  For example, `{"node": "11"}` in the following code example.
- Define DataFrame index columns for the labels.
  All DataFrame index columns except for the sample-time index column are automatically converted into labels for the respective table items.
  > **Note:** If you wish to use regular columns in your DataFrames as TSDB labels, convert these columns to index columns.
  > The following example converts the `symbol` and `exchange` columns to index columns that will be used as TSDB labels (in addition to the `time` index column):<br>
  > ```python
  > df.index.name="time"                              # Ensure that the sample-time index column is named "time"
  > df.reset_index(level=0, inplace=True)             # Reset the DataFrame indexes
  > df = df.set_index(["time", "symbol", "exchange"]) # Convert the "time" column and additional TSDB-label columns to DataFrame indexes
  > ```

In [11]:
# Prepare metric samples to ingets to the TSDB table
import numpy as np
from datetime import datetime, timedelta

end = datetime.now().replace(minute=0, second=0, microsecond=0)
rng = pd.date_range(end=end, periods=60, freq="300s", tz="EST")
df = pd.DataFrame(np.random.randn(len(rng), 3), index=rng, columns=["cpu", "mem", "disk"])
df = df.cumsum()
print(df.info(), df.head())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2019-09-15 08:05:00-05:00 to 2019-09-15 13:00:00-05:00
Freq: 300S
Data columns (total 3 columns):
cpu     60 non-null float64
mem     60 non-null float64
disk    60 non-null float64
dtypes: float64(3)
memory usage: 1.9 KB
None                                 cpu       mem      disk
2019-09-15 08:05:00-05:00 -1.115736 -1.229851  0.463388
2019-09-15 08:10:00-05:00 -0.113683 -0.515424  0.852152
2019-09-15 08:15:00-05:00 -1.603768 -1.155131  0.607183
2019-09-15 08:20:00-05:00 -1.458350 -0.673800  0.109492
2019-09-15 08:25:00-05:00  1.006173 -1.245499 -0.011838


In [12]:
# Ingest data into the TSDB table
client.write(backend="tsdb", table=tsdb_table, dfs=df, labels={"node": "11"})

<a id="frames-tsdb-read"></a>
### Read from the TSDB Table

Use the `read` method of the Frames client with the `"tsdb"` backend to read data from your TSDB table (i.e., query the database).

In [13]:
# Read time-series aggregates from the TSDB table as a data stream; use concat to assemble the DataFrames
query_str= "select avg(*), max(*), min(*) from '" + tsdb_table + "'"
tsdf = client.read(backend="tsdb", query=query_str, step="60m", start="now-7d", end="now", multi_index=True)
tsdf.head()

,,avg(cpu),avg(disk),avg(mem),max(cpu),max(disk),max(mem),min(cpu),min(disk),min(mem)
time,node,,,,,,,,,
2019-09-15 12:20:58,11,-1.072884,0.508054,-0.893552,-0.113683,0.852152,-0.515424,-1.603768,0.109492,-1.229851


<a id="frames-tsdb-delete"></a>
### Delete the TSDB Table

Use the `delete` method of the Frames client with the `"tsdb"` backend to delete the TSDB table that was used in the previous steps.

In [14]:
client.delete("tsdb", tsdb_table)

<a id='frames-stream'></a>
## Working with Streams ("stream" Backend)

The platform supports streams that have an AWS Kinesis-like API. For more information, see the [platform documentation](https://www.iguazio.com/docs/concepts/latest-release/streams/).

- [Initialization](#frames-stream-init)
- [Create a Stream](#frames-stream-create)
- [Write to the Stream](#frames-stream-write)
  - [Use the Write Method to Perform a Batch Update](#frames-stream-write-batch-update)
  - [Use the Execute Method's Put Command to Update a Single Record](#frames-stream-execute-put)
- [Read from the Stream](#frames-stream-read)
- [Delete the Stream](#frames-tsdb-delete)

<a id="frames-stream-init"></a>
### Initialization

Start out by defining a stream-path variable that will be used in the tutorial's code examples.<br>
The stream path (`strm`) is relative to the configured parent data container; see [Create a Stream](#frames-stream-create).

In [15]:
# Relative path to the stream within the parent platform data container
strm = os.path.join(os.getenv("V3IO_USERNAME") + "/examples/somestream")

<a id="frames-stream-create"></a>
### Create a Stream

Use the `create` method of the Frames client with the `"stream"` backend to create a new data stream.<br>
The mandatory `table` parameter specifies the relative stream path within the data container that was configured for the Frames client (see the [main initialization](#frames-init) step).
In the following example, the relative stream path is set by using the `strm` variable that was defined in the ["stream" backend initialization](#frames-stream-init) step.<br>
You can optionally use the `attrs` parameter to provide additional arguments.
For example, you can set the `shards` argument to the number of shards in the stream, or you can set the `retention_hours` argument to the stream's retention period in hours.

In [16]:
# Create a new stream
client.create(backend="stream", table=strm, attrs={"retention_hours": 48, "shards": 1})

<a id="frames-stream-write"></a>
### Write to the Stream

You can use either of the following methods to ingest data into your stream:

- [Use the Write Method to Perform a Batch Update](#frames-stream-write-batch-update)
- [Use the Execute Method's Put Command to Update a Single Record](#frames-stream-execute-put)

<a id="frames-stream-write-batch-update"></a>
#### Use the Write Method to Perform a Batch Update

Use the `write` method of the Frames client with the `"stream"` backend to ingest multiple records into your stream (batch update), as demonstrated in the following example.<br>
The `dfs` parameter can be set either to a single DataFrame (as done in the following example) or to multiple DataFrames &mdash; either as a DataFrame iterator or as a list of DataFrames.

In [17]:
# Prepare the ingestion data
end = datetime.now().replace(minute=0, second=0, microsecond=0)
rng = pd.date_range(end=end, periods=60, freq="300s", tz="Israel")
df = pd.DataFrame(np.random.randn(len(rng), 3), index=rng, columns=["cpu", "mem", "disk"])

# Ingest data into the stream
client.write("stream", table-strm, dfs=df)

<a id="frames-stream-execute-put"></a>
#### Use the Execute Method's Put Command to Update a Single Record

Use the `put` command of the `execute` method of the Frames client with the `"stream"` backend to add a single record to a stream.<br>
Use the `args` parameter of the `put` command to provide the necessary information:
set the mandatory `data` argument to the ingested record data.
You can optionally set the `clientinfo` argument to additional metadata and the `partition` argument to to a partition key; records with the same partition key are assigned to the same shard.

In [18]:
client.execute("stream", strm, "put", args={'data': "abcd", "clientinfo": "123"})

""


<a id="frames-stream-read"></a>
### Read from the Stream

Use the `read` method of the Frames client with the `"stream"` backend to read data from your stream.<br>
The mandatory `seek` parameter specifies the seek method, which determines the location within the target stream shard from which to read; some methods require setting additional parameters:

- `"earliest"` &mdash; start from the earliest point in the shard; (no additional parameters).
- `"latest"` &mdash; start from the latest location in the shard (i.e., consume only new records).
- `"time"` &mdash; start from a specific point in time, as specified in the `start` parameter (for example, `start="now-1d"`).
- `"sequence"` &mdash; start from a specific record sequence number, as specified in the `sequence` parameter (for example, `sequence=45`).

The `read` method can return a single DataFrame (default) or a DataFrame iterator (a stream) if the `iterator` parameter is set to `True`, as demonstrated in the following example.

In [19]:
# Read from the from the earliest available location (seek="earliest") in the first stream shard (shard_id=0);
# return the result as a DataFrame iterator (iterator=True) and iterate and print the returned data
dfs = client.read("stream", strm, seek="earliest", shard_id="0", iterator=True)
for df in dfs:
    print(df.head(4))

                 cpu      disk               index-0       mem raw_data  \
seq_number                                                                
1          -0.448749 -0.080189  2019-09-15T05:05:00Z -0.128496            
2          -0.255004 -1.169343  2019-09-15T05:10:00Z -0.858265            
3          -0.469349  0.844941  2019-09-15T05:15:00Z  0.994743            
4           1.062668 -1.448676  2019-09-15T05:20:00Z -1.243823            

                             stream_time  
seq_number                                
1          2019-09-15 13:21:56.969021708  
2          2019-09-15 13:21:56.969021708  
3          2019-09-15 13:21:56.969021708  
4          2019-09-15 13:21:56.969021708  


<a id="frames-tsdb-stream"></a>
### Delete the Stream

Use the `delete` method of the Frames client with the `"stream"` backend to delete the TSDB table that was used in the previous steps.

In [20]:
client.delete("stream", strm)

<a id="frames-cleanup"></a>
## Cleanup

You can optionally delete any of the directories or files that you created.
See the instructions in the [Creating and Deleting Container Directories](https://www.iguazio.com/docs/tutorials/latest-release/getting-started/containers/#create-delete-container-dirs) tutorial.
For example, the following code uses a local file-system command to delete the entire **&lt;running user&gt;/examples/** directory in the "users" container.
Edit the path, as needed, then remove the comment mark (`#`) and run the code.

In [ ]:
#!rm -rf /User/examples/